# 4. 작사가 인공지능 만들기

- 데이터 다듬기
- 인공지능 학습시키기
- 잘 만들어졌는지 평가하기

In [1]:
import os
import re    
import glob
import numpy as np         
import tensorflow as tf    
from sklearn.model_selection import train_test_split

## 데이터 다듬기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


데이터의 전처리

In [32]:
#특수문자 제거 및 공백처리
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    corpus.append(preprocess_sentence(sentence))
    
print(len(corpus))

187088


In [5]:
for idx, sentence in enumerate(raw_corpus):
    print(len(sentence.split()))
    if idx > 5: break

8
8
9
4
4
6
6


데이터 정제

In [33]:
corpus = [] #정제한 문장 담기

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    
    #정제하고 담기
    sentence = preprocess_sentence(sentence)
    
    if len(sentence.split()) >= 16: continue
    corpus.append(sentence)
        
len(corpus)

156227

In [30]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  258  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f9b61461850>


In [8]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

src_input.shape

[   2   50    4   95  303   62   53    9  946 6269    3    0    0    0]
[  50    4   95  303   62   53    9  946 6269    3    0    0    0    0]


(156227, 14)

평가 데이터셋 분리

In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=7)

In [34]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE #한 번에 다 넣을 수 없어서 나눠서 넣어준다(256 임의로 나눠서)

VOCAB_SIZE = tokenizer.num_words + 1   
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder= False)
print(len(train_dataset))

489


In [11]:
validation_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder= True)

print(len(validation_dataset))

122


In [12]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


## 인공지능 학습시키기

In [26]:
#모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #문자를 기계가 이해할 수 있는 숫자로 바꾼 결과 또는 과정
hidden_size = 3000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [20]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.34693808e-04, -8.46081603e-05,  1.72471686e-04, ...,
         -3.21308791e-04, -5.51694866e-05,  8.17072359e-05],
        [ 2.99747859e-04, -7.94460811e-06,  3.66770517e-04, ...,
         -4.69062303e-04,  4.45991791e-05,  1.84783319e-04],
        [ 1.83238662e-04,  6.91709574e-05,  5.45984542e-04, ...,
         -4.96059249e-04,  2.09877835e-04,  9.61716432e-05],
        ...,
        [ 1.45501911e-03, -3.10460280e-04, -3.31074320e-04, ...,
         -6.83339662e-04, -6.08452945e-04,  2.06974288e-03],
        [ 1.73379981e-03, -4.50003921e-04, -3.63521453e-04, ...,
         -6.63492305e-04, -7.74905842e-04,  2.36349413e-03],
        [ 1.99869834e-03, -5.57790627e-04, -3.82129452e-04, ...,
         -6.56590972e-04, -9.12524527e-04,  2.58376123e-03]],

       [[ 1.34693808e-04, -8.46081603e-05,  1.72471686e-04, ...,
         -3.21308791e-04, -5.51694866e-05,  8.17072359e-05],
        [ 4.64207027e-04,  1.99271206e-04,  1

In [22]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  39084000  
_________________________________________________________________
lstm_3 (LSTM)                multiple                  72012000  
_________________________________________________________________
dense_1 (Dense)              multiple                  36015001  
Total params: 150,183,257
Trainable params: 150,183,257
Non-trainable params: 0
_________________________________________________________________


In [24]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

In [25]:
model.fit(train_dataset,
         epochs=10,
         validation_data=validation_dataset)

Epoch 1/10
489/489 [==============================] - 497s 1s/step - loss: 3.2281 - val_loss: 2.8956
Epoch 2/10
489/489 [==============================] - 499s 1s/step - loss: 2.7001 - val_loss: 2.6351
Epoch 3/10
489/489 [==============================] - 500s 1s/step - loss: 2.3529 - val_loss: 2.4448
Epoch 4/10
489/489 [==============================] - 501s 1s/step - loss: 2.0051 - val_loss: 2.3020
Epoch 5/10
489/489 [==============================] - 502s 1s/step - loss: 1.6796 - val_loss: 2.2077
Epoch 6/10
489/489 [==============================] - 503s 1s/step - loss: 1.4089 - val_loss: 2.1577
Epoch 7/10
489/489 [==============================] - 503s 1s/step - loss: 1.2096 - val_loss: 2.1534
Epoch 8/10
489/489 [==============================] - 503s 1s/step - loss: 1.0921 - val_loss: 2.1643
Epoch 9/10
489/489 [==============================] - 502s 1s/step - loss: 1.0324 - val_loss: 2.1833
Epoch 10/10
489/489 [==============================] - 503s 1s/step - loss: 1.0006 - val_lo

## 잘 만들어졌는지 평가하기

In [27]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True:
        predict = model(test_tensor)  
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated 

In [28]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [29]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love memory memory nerves difficult difficult difficult thumbs thumbs until until bumping punched attention attention suggestion suggestion suggestion '

## 회고

데이터 전처리 과정에서 특수문자제거, 

val_loss가 2.2 이하가 나와야 한다고 했다.

lms노드에 적혀 있는 것 처럼 val_loss가 2.2이면 hidden_size와 embedding_size의 크기를 조절해야한다고 했다.

여러 시도를 해봤을 때, embedding_size와 hidden_size가 val_loss에 영향을 끼친다는 사실을 발견했고 embedding_size보다는 hidden_size가 더 커질 때 val_loss가 적게 나온다는 사실을 확인했다.

중간에 val_loss가 왜 높아지는지는 아직 잘 모르겠으나 이건 앞으로 알아가야할 방향인 것 같다.
